# Index Return data

- using yahoo finance for historical stock return data 
- going to need the following
    - Dates of the announcement
    - Index returns 10 days before the announcement
    - Index returns the day of the announcement
    - Index returns 10 days after the announcement 

In [6]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [11]:

dates = pd.read_csv('dates/dates_updated.csv')
dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])
dates['document_type'] = 'statement' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Statement Date'].min() - pd.Timedelta(days=15)
end_date = dates['Statement Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Statement Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

statements_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
statements_df = statements_df[column_order]

statements_df

statements_df.to_csv('raw_data/statement_prices.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_9312/484069958.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates['Statement Date'] = pd.to_datetime(dates['Statement Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*********************

### Problem
Need to figure out how to get the days for around the intermeeting dates now because 
the dates in the html links are the same as the fed statements for those, which would cause a problem 
when getting return data.

How to get around this?
Find a dataset with the dates of the intermeeting or make our own (just manually do it - would take an hour maybe)

Ended up just manually doing this

In [13]:

dates = pd.read_csv('dates/dates_updated.csv')
dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
dates['document_type'] = 'intermeeting' 

tickers = [
    '^GSPC',     # S&P 500
    '^IXIC',     # NASDAQ Composite
    '^DJI',      # Dow Jones Industrial Average
    '^RUT',      # Russell 2000
    '^W5000',    # Wilshire 5000
    'XLF',       # Financials Sector (ETF)
    'XLRE',      # Real Estate Sector (ETF)
    'XLU',       # Utilities Sector (ETF)
    'XLY',       # Consumer Discretionary Sector (ETF)
    'XLP',       # Consumer Staples Sector (ETF)
    'XLE',       # Energy Sector (ETF)
    'XLV',       # Healthcare Sector (ETF)
    'XLI',       # Industrials Sector (ETF)
    'XLB',       # Materials Sector (ETF)
    'XLK',       # Information Technology Sector (ETF)
    'XLC',       # Communication Services Sector (ETF)
    '^IRX',      # Three-month Treasury Bill Yield
    '^TNX',      # Ten-year Treasury Yield
]

start_date = dates['Intermeeting Date'].min() - pd.Timedelta(days=15)
end_date = dates['Intermeeting Date'].max() + pd.Timedelta(days=15)

all_indices_data = {}
for ticker in tickers:
    #print(f"Downloading data for {ticker}...")
    data = yf.download(ticker, start=start_date, end=end_date)
    data.columns = data.columns.get_level_values(0)
    data['return'] = data['Close'].pct_change()
    all_indices_data[ticker] = data[['return']].dropna()
    #print(f"Data for {ticker} downloaded.")

rows = []

for index, row_fomc in dates.iterrows():
    date = row_fomc['Intermeeting Date']
    document_type = row_fomc['document_type']
    for ticker in tickers:
        row = {'announcement_date': date, 'ticker': ticker, 'document_type': document_type} 
        for t in range(-15, 16):
            target_date = date + pd.Timedelta(days=t)
            if target_date in all_indices_data[ticker].index:
                row[f'T{t:+}'] = all_indices_data[ticker].loc[target_date, 'return']
            else:
                row[f'T{t:+}'] = pd.NA
        rows.append(row)

intermeeting_df = pd.DataFrame(rows)

column_order = ['announcement_date', 'ticker', 'document_type'] + [f'T{t:+}' for t in range(-15, 16)]
intermeeting_df = intermeeting_df[column_order]

intermeeting_df

intermeeting_df.to_csv('raw_data/intermeeting_prices.csv', index=False)

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_9312/3896584411.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates['Intermeeting Date'] = pd.to_datetime(dates['Intermeeting Date'])
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%**************

### Need to merge the two datasets

In [10]:
final_df

,announcement_date,ticker,document_type,T-15,T-14,T-13,T-12,T-11,T-10,T-9,...,T+6,T+7,T+8,T+9,T+10,T+11,T+12,T+13,T+14,T+15
0,2000-03-23,^GSPC,intermeeting,<NA>,0.025602,-0.004723,<NA>,<NA>,-0.008207,-0.017685,...,0.000524,-0.013656,0.007164,<NA>,<NA>,0.004931,-0.007464,-0.004924,0.009392,0.009998
1,2000-03-23,^IXIC,intermeeting,<NA>,0.030548,0.000349,<NA>,<NA>,-0.028004,-0.04088,...,-0.039145,-0.040214,0.025783,<NA>,<NA>,-0.076353,-0.017707,0.0049,0.023587,0.041919
2,2000-03-23,^DJI,intermeeting,<NA>,0.015644,-0.008182,<NA>,<NA>,0.001844,-0.013661,...,0.007554,-0.003491,-0.005312,<NA>,<NA>,0.027469,-0.005087,-0.011726,0.007282,-0.000251
3,2000-03-23,^RUT,intermeeting,<NA>,0.01912,-0.003696,<NA>,<NA>,-0.02264,-0.029061,...,-0.028692,-0.02105,0.014147,<NA>,<NA>,-0.042757,-0.019223,0.023552,0.027913,0.0197
4,2000-03-23,^W5000,intermeeting,<NA>,0.024762,-0.00388,<NA>,<NA>,-0.01357,-0.021922,...,-0.008538,-0.01743,0.011479,<NA>,<NA>,-0.012209,-0.011746,-0.000518,0.014204,0.013746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3703,2025-04-09,XLB,intermeeting,-0.000231,0.001849,-0.000461,-0.018352,<NA>,<NA>,0.010935,...,-0.007856,-0.007794,0.006858,<NA>,<NA>,<NA>,-0.015232,0.02339,0.001229,<NA>
3704,2025-04-09,XLK,intermeeting,0.003308,-0.022304,-0.009228,-0.024254,<NA>,<NA>,0.000484,...,0.003647,-0.034745,-0.005415,<NA>,<NA>,<NA>,-0.025822,0.024377,0.028993,<NA>
3705,2025-04-09,XLC,intermeeting,0.012666,-0.008505,-0.00444,-0.029498,<NA>,<NA>,0.007416,...,0.002503,-0.026924,0.006471,<NA>,<NA>,<NA>,-0.022503,0.026083,0.016026,<NA>
3706,2025-04-09,^IRX,intermeeting,0.0,0.001913,-0.000477,0.0,<NA>,<NA>,0.001194,...,0.001192,0.001905,-0.000713,<NA>,<NA>,<NA>,0.001189,0.000713,-0.001187,<NA>


# Sentiment Analysis 